## SVM Focus

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
cosine_sim_feaure.gz
sample.rtf
sample_submission.csv
test.csv
test_with_cosine_v0.csv
train.csv
train_with_cosine_v0.csv



## Extract, Transform, Load

In [3]:
from sklearn.model_selection import train_test_split

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")

df_submit = read_data("input/test.csv")
df_submit = df_submit.reset_index(drop=True)
df_submit = df_submit.reset_index()
df_submit = df_submit.rename(columns = {'index': 'mat_idx'})


# Print the column names
print (df_train.columns.values.tolist())

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)
Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345796, 3)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate']


## Training the Model

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Read in cosine distance feature
try:
    df_train = pd.read_csv("input/train_with_cosine_v0.csv")
except OSError:
    df_train['cos_sim'] = df_train.apply(lambda row: gen_cosine_sim(row['question1'],
                                                                    row['question2'],
                                                                    vectorizer), axis=1)

    
# cos_feature_vec.T[0].tolist()

print("Cosine Similarity Feature Generated...")

scaler = MinMaxScaler().fit(df_train[['cos_sim']])
X = scaler.transform(df_train[['cos_sim']])

y = df_train['is_duplicate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=37)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

Cosine Similarity Feature Generated...


((323430, 1), (80858, 1), (323430,), (80858,))

In [42]:
print("Creating the bag of words feature")
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

# Use this as "stop_words" arg if include stopwords
# stopwords.words('english')

# Initialize CountVectorizer
# For now assume stop-words are not discounted. No terms are thresholded. 
vectorizer = CountVectorizer(analyzer = stemmed_words,   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             ngram_range = (1,2), \
                             max_features = 100) 


from sklearn.metrics.pairwise import cosine_similarity

# cos_feature_vec = np.zeros((df_train.shape[0],1))

def gen_cosine_sim(s1, s2, vectorizer):
    try:
        features = vectorizer.fit_transform([s1, s2])
        similarities = cosine_similarity(features)
        return similarities[0,1]
    except:
        return 0.5 # May need to change for normalization 


Creating the bag of words feature


## Train Random Forest

In [38]:
clf = RandomForestClassifier(n_estimators=50, max_features=None, max_depth=None, bootstrap=True)
# Train the random forest on all test data. CV is "for free"
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [39]:
## Test on Randomly selected Subset

In [48]:
from random import randint

## SET UP TEST INDICES TO SELECT RANDOM ROWS

TEST_SAMPLES = 100
indices = np.arange(TEST_SAMPLES)
for i in range(len(indices)):
    indices[i] = randint(0, df_submit.shape[0])
    
df_sampled = df_submit.loc[indices]

df_sampled['cos_sim'] = df_sampled.apply(lambda row: gen_cosine_sim(row['question1'],
                                                                    row['question2'],
                                                                    vectorizer), axis=1)

print("Cosine Similarity Feature Generated...")

scaler = MinMaxScaler().fit(df_sampled[['cos_sim']])
X_samp = scaler.transform(df_sampled[['cos_sim']])
probs = clf.predict_proba(X_samp)





[1496847 1854244  527456  633537 2070713 2318766  484410 2218147  519297
 1266868  156878  628787 1838637 2109181 1682587    3186 1848609   68665
 1937438  147194  521835 1180955  226716  661779 1623775 1173823  357365
  661079 1245020  927692  125427 1619504 1594839 1996327 2310637 1316493
  552579  428626  221499 2225121 1382084 1786007 1352745   45639 1494487
  616623 1175415 1174769 1243330   86182  969270 1614532 1236523  526358
 1931911 1602518 1060431  840544  821139  381314 1970811 1223046  383802
 1589607 1801806 1190989 1812180  782104 2225733   28105  425658  493550
 1142444 2214993 1320990 2030804 2180886  231504  930046   77309 1629625
 1567111  843557  115393 1937078  876456    4959 1923015  483821  516618
 1882875  504215  632594 2329856  152938 1229311  853186 1640866 1426874
 1525889]
Cosine Similarity Feature Generated...


In [51]:
for idx, p in enumerate(probs):
    print(p)
    print("%s\n%s\n" % (df_submit.loc[indices[idx]]['question1'], df_submit.loc[indices[idx]]['question2']))

[ 1.  0.]
How what do you identify your competition?
How do I identify my core hell?

[ 0.01921651  0.98078349]
What performing the corporate culture like at Netflix? How is the culture different than other companies?
What is the is corporate culture like at Cabot? How is the culture different than other companies?

[ 0.43228244  0.56771756]
How can improve my English speaking backward?
How siberia I improve English speaking skills?

[ 1.  0.]
I don't want to sleep.?
Why when we don't sleep much for a couple of days we start to ocw shake and our heart starts beating really fast?

[ 0.90470286  0.09529714]
How much does an independent artist make on a iTunes track sale?
How much does an typically make each time their song is played on the radio?

[ 1.  0.]
Why is The Walking indonesia picture so grainy even in 1080p?
What the difference between 720p HEVC and 720p normal movies?

[ 0.9452565  0.0547435]
Which is the favourite sex position of indian girls?
What is your favorite position d